In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [ ]:
import pandas as pd

# Load the datasets
#Step 1: Loading the Dataset
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
#2. Explore the Data
# View basic info and statistics
train_data = pd.read_csv('train.csv')
print(train_data.info())
print(train_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [ ]:
test_data = pd.read_csv('test.csv')
print(test_data.info())
print(test_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [ ]:
#Handling the missing value
# Check for missing values
print(train_data.isnull().sum())
print(test_data.isnull().sum())

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64
Id                 0
MSSubClass         0
MSZoning           4
LotFrontage      227
LotArea            0
                ... 
MiscVal            0
MoSold             0
YrSold             0
SaleType           1
SaleCondition      0
Length: 80, dtype: int64


In [ ]:
# Fill missing values with median for numerical features
# Fill missing values with median for numerical features ONLY
for column in train_data.select_dtypes(include=['number']):
    train_data[column].fillna(train_data[column].median(), inplace=True)

for column in test_data.select_dtypes(include=['number']):
    test_data[column].fillna(test_data[column].median(), inplace=True)

In [ ]:
# Check for missing values
print(train_data.isnull().sum())
print(test_data.isnull().sum())

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64
Id               0
MSSubClass       0
MSZoning         4
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 80, dtype: int64


In [ ]:
# Fill missing values with mode (most frequent value) for categorical features
train_data.fillna(train_data.mode().iloc[0], inplace=True)
test_data.fillna(test_data.mode().iloc[0], inplace=True)


In [ ]:
#4.Detect and Handle Outliers
# Calculate IQR for numerical features ONLY
Q1 = train_data.select_dtypes(include=['number']).quantile(0.25)
Q3 = train_data.select_dtypes(include=['number']).quantile(0.75)
IQR = Q3 - Q1

# Define outlier condition for numerical features
outlier_condition = ((train_data.select_dtypes(include=['number']) < (Q1 - 1.5 * IQR)) | (train_data.select_dtypes(include=['number']) > (Q3 + 1.5 * IQR)))

# Remove outliers
train_data = train_data[~outlier_condition.any(axis=1)]


In [ ]:

from sklearn.ensemble import IsolationForest

# Identify numerical features
numerical_features = train_data.select_dtypes(include=['float64', 'int64']).columns

# Fit Isolation Forest
iso_forest = IsolationForest(contamination=0.01)
outliers = iso_forest.fit_predict(train_data[numerical_features])

# Remove outliers
train_data = train_data[outliers != -1]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [ ]:
#5. Encode Categorical Variables
from sklearn.preprocessing import LabelEncoder

# Identify categorical features
categorical_features = train_data.select_dtypes(include=['object']).columns

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to categorical features
for feature in categorical_features:
    # Fit on the combined unique values from both datasets
    unique_values = list(train_data[feature].unique()) + list(test_data[feature].unique())
    label_encoder.fit(unique_values)

    train_data[feature] = label_encoder.transform(train_data[feature])
    test_data[feature] = label_encoder.transform(test_data[feature])

In [ ]:
# Apply One-Hot Encoding
train_data = pd.get_dummies(train_data, columns=categorical_features, drop_first=True)
test_data = pd.get_dummies(test_data, columns=categorical_features, drop_first=True)

# Align the train and test datasets (in case they have different columns after one-hot encoding)
test_data = test_data.reindex(columns=train_data.columns, fill_value=0)


In [ ]:
#6. Feature Scaling
# Check if 'feature1' exists in the DataFrames
print("feature1 in train_data:", 'feature1' in train_data.columns)
print("feature1 in test_data:", 'feature1' in test_data.columns)

# If 'feature1' doesn't exist, identify the new columns created from one-hot encoding
if 'feature1' not in train_data.columns:
    feature1_onehot_cols = [col for col in train_data.columns if 'feature1' in col]
    print("One-hot encoded columns for feature1:", feature1_onehot_cols)

# Assuming 'feature1' was one-hot encoded, adjust the interaction calculation
if feature1_onehot_cols:
    train_data['interaction_feature'] = train_data[feature1_onehot_cols[0]] * train_data['feature2']
    test_data['interaction_feature'] = test_data[feature1_onehot_cols[0]] * test_data['feature2']
else:
    print("Neither 'feature1' nor its one-hot encoded columns were found. Please check for typos or previous data manipulations.")


feature1 in train_data: False
feature1 in test_data: False
One-hot encoded columns for feature1: []
Neither 'feature1' nor its one-hot encoded columns were found. Please check for typos or previous data manipulations.


In [ ]:
#7. Feature Engineering
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(train_data[numerical_features])

# Convert polynomial features to DataFrame and append to original data
poly_features_df = pd.DataFrame(poly_features, columns=poly.get_feature_names_out(numerical_features))
train_data = pd.concat([train_data, poly_features_df], axis=1)

In [ ]:
#7. Feature Engineering
# Separate features and target variable
target_column = 'SalePrice'  # Replace with the actual name of your target column
X = train_data.drop(target_column, axis=1)
y = train_data[target_column]

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)